In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

In [ ]:
obj = vg.ValidationGeneral(lf, "test data")

In [ ]:
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

In [ ]:
lfall.with_columns("LESION", "TELEPHONE NO").filter(
  (pl.col("LESION") == True)
  & ~(pl.col("TELEPHONE NO").str.contains(r"^(6?0[1-9])\d{7,9}$"))
).collect()
